## Telecom Project

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import sqlalchemy
from ydata_profiling import ProfileReport
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import roc_auc_score, accuracy_score
from lightgbm import LGBMClassifier
import lightgbm as lgb
import optuna
import torch


RANDOM_STATE = 22623

### SQL Connect

In [2]:
db_config = {
    'user': 'praktikum_student',
    'pwd': 'Sdf4$2;d-d30pp',
    'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
    'port': 6432,
    'db': 'data-science-final'
}
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(
    db_config['user'],
    db_config['pwd'],
    db_config['host'],
    db_config['port'],
    db_config['db'],
)

In [3]:
engine = sqlalchemy.create_engine(connection_string);

In [4]:
query = '''
SELECT *
FROM telecom.contract
'''
df_contract = pd.read_sql_query(query, con=engine)
#df_contract.columns = df_contract.iloc[0]
#df_contract = df_contract[1:]

In [5]:
query = '''
SELECT *
FROM telecom.personal
'''
df_personal = pd.read_sql_query(query, con=engine)
# df_personal.columns = df_personal.iloc[0]
# df_personal = df_personal[1:]

In [6]:
query = '''
SELECT *
FROM telecom.internet
'''
df_internet = pd.read_sql_query(query, con=engine)
df_internet.columns = df_internet.iloc[0]
df_internet = df_internet[1:]

In [7]:
query = '''
SELECT *
FROM telecom.phone
'''
df_phone = pd.read_sql_query(query, con=engine)
df_phone.columns = df_phone.iloc[0]
df_phone = df_phone[1:]

### EDA

In [8]:
df_contract.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   BeginDate         7043 non-null   object 
 2   EndDate           1869 non-null   object 
 3   Type              7043 non-null   object 
 4   PaperlessBilling  7043 non-null   object 
 5   PaymentMethod     7043 non-null   object 
 6   MonthlyCharges    7043 non-null   float64
 7   TotalCharges      7032 non-null   float64
dtypes: float64(2), object(6)
memory usage: 440.3+ KB


In [9]:
df_contract = df_contract[df_contract.TotalCharges.notna()]

In [10]:
df_contract.head()

,customerID,BeginDate,EndDate,Type,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges
2,3668-QPYBK,2018-08-09,2019-12-01,Month-to-month,Yes,Mailed check,53.85,108.15
4,9237-HQITU,2019-01-26,2019-11-01,Month-to-month,Yes,Electronic check,70.70,151.65
5,9305-CDSKC,2018-12-26,2019-11-01,Month-to-month,Yes,Electronic check,99.65,820.50
8,7892-POOKP,2019-04-27,2019-11-01,Month-to-month,Yes,Electronic check,104.80,3046.05
12,0280-XJGEX,2018-11-13,2019-10-01,Month-to-month,Yes,Bank transfer (automatic),103.70,5036.30


In [11]:
df_internet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5517 entries, 1 to 5517
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   customerID        5517 non-null   object
 1   InternetService   5517 non-null   object
 2   OnlineSecurity    5517 non-null   object
 3   OnlineBackup      5517 non-null   object
 4   DeviceProtection  5517 non-null   object
 5   TechSupport       5517 non-null   object
 6   StreamingTV       5517 non-null   object
 7   StreamingMovies   5517 non-null   object
dtypes: object(8)
memory usage: 344.9+ KB


In [12]:
ProfileReport(df_internet).to_notebook_iframe()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

В принципе, ничего особо примечательного. Все признаки распределены более или менее равномерно, мультиколлинеарности на первый взгляд нет.  

In [13]:
df_personal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   customerID     7043 non-null   object
 1   gender         7043 non-null   object
 2   SeniorCitizen  7043 non-null   int64 
 3   Partner        7043 non-null   object
 4   Dependents     7043 non-null   object
dtypes: int64(1), object(4)
memory usage: 275.2+ KB


In [14]:
ProfileReport(df_personal).to_notebook_iframe()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Тут всё довольно стандартно. Пенсионеров в 6 раз меньше, чем взрослых, а наличие партнёра заметно коррелирует с наличием детей.

In [15]:
len(df_personal) == len(df_contract)

False

In [16]:
df_phone.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6361 entries, 1 to 6361
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   customerID     6361 non-null   object
 1   MultipleLines  6361 non-null   object
dtypes: object(2)
memory usage: 99.5+ KB


In [17]:
ProfileReport(df_phone).to_notebook_iframe()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Здесь тоже ничего супер интересного или важного.

### Merge + extra EDA

In [18]:
df_merge = df_personal.merge(df_contract)

In [19]:
df_merge = df_merge.merge(df_internet, how='left')
df_merge = df_merge.merge(df_phone, how='left')

In [20]:
df_merge.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,BeginDate,EndDate,Type,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,MultipleLines
0,7590-VHVEG,Female,0,Yes,No,2019-04-29,None,Month-to-month,Yes,Electronic check,29.85,29.85,DSL,No,Yes,No,No,No,No,NaN
1,5575-GNVDE,Male,0,No,No,2019-03-26,None,One year,No,Mailed check,56.95,1889.50,DSL,Yes,No,Yes,No,No,No,No
2,3668-QPYBK,Male,0,No,No,2018-08-09,2019-12-01,Month-to-month,Yes,Mailed check,53.85,108.15,DSL,Yes,Yes,No,No,No,No,No
3,7795-CFOCW,Male,0,No,No,2018-12-22,None,One year,No,Bank transfer (automatic),42.30,1840.75,DSL,Yes,No,Yes,Yes,No,No,NaN
4,9237-HQITU,Female,0,No,No,2019-01-26,2019-11-01,Month-to-month,Yes,Electronic check,70.70,151.65,Fiber optic,No,No,No,No,No,No,No


In [21]:
target = df_merge['EndDate']

In [22]:
df_merge = df_merge.loc[ : , df_merge.columns!='EndDate'].fillna('No')

In [23]:
df_merge['target'] = target

In [24]:
df_merge = df_merge.replace({'No': 0, 'Yes': 1})
df_merge = df_merge.replace({'Female':0, 'Male':1})

In [25]:
df_merge = df_merge.drop('BeginDate', axis=1)
df_merge = df_merge.drop('customerID', axis=1)
#df_merge['BeginDate'] = pd.to_datetime(df_merge['BeginDate'])
#df_merge['EndDate'] = pd.to_datetime(df_merge['EndDate'])

In [26]:
df_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7032 entries, 0 to 7031
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   gender            7032 non-null   int64  
 1   SeniorCitizen     7032 non-null   int64  
 2   Partner           7032 non-null   int64  
 3   Dependents        7032 non-null   int64  
 4   Type              7032 non-null   object 
 5   PaperlessBilling  7032 non-null   int64  
 6   PaymentMethod     7032 non-null   object 
 7   MonthlyCharges    7032 non-null   float64
 8   TotalCharges      7032 non-null   float64
 9   InternetService   7032 non-null   object 
 10  OnlineSecurity    7032 non-null   int64  
 11  OnlineBackup      7032 non-null   int64  
 12  DeviceProtection  7032 non-null   int64  
 13  TechSupport       7032 non-null   int64  
 14  StreamingTV       7032 non-null   int64  
 15  StreamingMovies   7032 non-null   int64  
 16  MultipleLines     7032 non-null   int64  


In [27]:
df_merge = df_merge.merge(pd.get_dummies(df_merge[['Type', 'PaymentMethod', 'InternetService']], drop_first=True), left_index=True, right_index=True)

In [28]:
df_merge = df_merge.drop(['Type', 'PaymentMethod', 'InternetService'], axis=1);

In [29]:
df_merge.target = df_merge.target.notnull()

### Splitting data

In [30]:
x_train1, x_test, y_train1, y_test = train_test_split(df_merge.drop('target', axis=1), df_merge.target, train_size=0.8, random_state=RANDOM_STATE)
x_train, x_valid, y_train, y_valid = train_test_split(x_train1, y_train1, train_size=0.75, random_state=RANDOM_STATE)

In [31]:
scaler = StandardScaler()

In [32]:
scaler.fit(x_train[['MonthlyCharges', 'TotalCharges']])
x_train[['MonthlyCharges', 'TotalCharges']] = scaler.transform(x_train[['MonthlyCharges', 'TotalCharges']])
x_test[['MonthlyCharges', 'TotalCharges']] = scaler.transform(x_test[['MonthlyCharges', 'TotalCharges']])
x_valid[['MonthlyCharges', 'TotalCharges']] = scaler.transform(x_valid[['MonthlyCharges', 'TotalCharges']])

In [33]:
x_train.head()

,gender,SeniorCitizen,Partner,Dependents,PaperlessBilling,MonthlyCharges,TotalCharges,OnlineSecurity,OnlineBackup,DeviceProtection,...,StreamingTV,StreamingMovies,MultipleLines,Type_One year,Type_Two year,PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,InternetService_DSL,InternetService_Fiber optic
6209,0,0,1,1,1,-0.225634,0.791305,0,1,1,...,1,1,0,1,0,0,0,0,1,0
899,0,0,0,0,0,1.102842,-0.609470,0,0,1,...,1,1,1,0,0,1,0,0,0,1
6546,1,0,0,0,0,-1.515821,-0.834432,0,0,0,...,0,0,0,1,0,0,0,1,0,0
6841,1,0,1,1,1,-0.109101,0.269097,0,1,1,...,0,0,0,0,0,0,0,1,1,0
621,1,0,1,1,1,0.661682,0.458640,0,0,1,...,1,0,0,1,0,0,1,0,0,1


### Gradient Boosting

In [34]:
model = LGBMClassifier()

In [35]:
model.fit(x_train, y_train)
pred = model.predict_proba(x_valid)

In [36]:
roc_auc_score(y_valid, pred[:, 1])

0.8480609931047249

In [37]:
#import optuna.integration.lightgbm as lgb

In [38]:
def objective(trial):
    dtrain = lgb.Dataset(x_train, label=y_train)
    param = {
        "boosting_type": trial.suggest_categorical("boosting_type", ['dart', 'gbdt']),
        "verbosity": -1,
        "objective": "binary",
        "metric": "auc",
        "max_depth": trial.suggest_int('max_depth', 2, 32),
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 4, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
    }

    gbm = lgb.train(param, dtrain)
    preds = gbm.predict(x_valid)
    #pred_labels = np.rint(preds)
    roc = roc_auc_score(y_valid, preds)
    return roc

In [39]:
dval = lgb.Dataset(x_test, y_test)

In [40]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=120)

[I 2023-06-30 22:30:06,155] A new study created in memory with name: no-name-6cd79b18-6303-444f-87af-5caea8af90ea
[I 2023-06-30 22:30:06,300] Trial 0 finished with value: 0.8501023236203022 and parameters: {'boosting_type': 'gbdt', 'max_depth': 17, 'lambda_l1': 3.575649826734775e-06, 'lambda_l2': 4.2448832438608845e-08, 'num_leaves': 211, 'feature_fraction': 0.7333201880870217, 'bagging_fraction': 0.7480952267491874, 'bagging_freq': 1, 'min_child_samples': 84}. Best is trial 0 with value: 0.8501023236203022.
[I 2023-06-30 22:30:06,466] Trial 1 finished with value: 0.8509018875879905 and parameters: {'boosting_type': 'dart', 'max_depth': 8, 'lambda_l1': 2.200254219705998e-05, 'lambda_l2': 5.9038981632690165e-05, 'num_leaves': 176, 'feature_fraction': 0.44977458715732865, 'bagging_fraction': 0.6244896295748656, 'bagging_freq': 7, 'min_child_samples': 50}. Best is trial 1 with value: 0.8509018875879905.
[I 2023-06-30 22:30:06,658] Trial 2 finished with value: 0.850395411376947 and paramet

In [41]:
model = LGBMClassifier(**study.best_params, n_estimators=70)

In [42]:
model.fit(x_train1, y_train1)

[LightGBM] [Warning] feature_fraction is set=0.9129760083482733, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9129760083482733
[LightGBM] [Warning] lambda_l1 is set=0.08573838922097844, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.08573838922097844
[LightGBM] [Warning] bagging_fraction is set=0.8812829072721462, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8812829072721462
[LightGBM] [Warning] lambda_l2 is set=0.9191248574209432, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.9191248574209432
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4


LGBMClassifier(bagging_fraction=0.8812829072721462, bagging_freq=4,
               boosting_type='dart', feature_fraction=0.9129760083482733,
               lambda_l1=0.08573838922097844, lambda_l2=0.9191248574209432,
               max_depth=29, min_child_samples=88, n_estimators=70,
               num_leaves=169)

In [43]:
study.best_trial

FrozenTrial(number=103, values=[0.8608617294234399], datetime_start=datetime.datetime(2023, 6, 30, 22, 30, 21, 377405), datetime_complete=datetime.datetime(2023, 6, 30, 22, 30, 21, 522460), params={'boosting_type': 'dart', 'max_depth': 29, 'lambda_l1': 0.08573838922097844, 'lambda_l2': 0.9191248574209432, 'num_leaves': 169, 'feature_fraction': 0.9129760083482733, 'bagging_fraction': 0.8812829072721462, 'bagging_freq': 4, 'min_child_samples': 88}, distributions={'boosting_type': CategoricalDistribution(choices=('dart', 'gbdt')), 'max_depth': IntUniformDistribution(high=32, low=2, step=1), 'lambda_l1': LogUniformDistribution(high=10.0, low=1e-08), 'lambda_l2': LogUniformDistribution(high=10.0, low=1e-08), 'num_leaves': IntUniformDistribution(high=256, low=4, step=1), 'feature_fraction': UniformDistribution(high=1.0, low=0.4), 'bagging_fraction': UniformDistribution(high=1.0, low=0.4), 'bagging_freq': IntUniformDistribution(high=7, low=1, step=1), 'min_child_samples': IntUniformDistributi

In [44]:
test_pred = model.predict_proba(x_test)
print('ROC AUC =', roc_auc_score(y_test, test_pred[:, 1]))

ROC AUC = 0.8175683606776045


In [45]:
print('Accuracy =', accuracy_score(y_test, np.rint(test_pred[:, 1])))

Accuracy = 0.7448471926083866


### FCN

In [46]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

In [47]:
class CustomDataset(Dataset):
    def __init__(self, x, y):
        self.x = torch.tensor(x, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

In [48]:
dataset = CustomDataset(x_train1.values, y_train1.values)
trainloader = DataLoader(dataset, batch_size=64, shuffle=True)

In [49]:
class BinaryClassifier(nn.Module):
    def __init__(self, input_size):
        super(BinaryClassifier, self).__init__()
        self.fc1 = nn.Linear(input_size, 32)
        self.fc2 = nn.Linear(32, 64)
        self.fc3 = nn.Linear(64, 1)
        self.drop = nn.Dropout(.1)
        self.batchnorm1 = nn.BatchNorm1d(32)
        self.batchnorm2 = nn.BatchNorm1d(64)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.batchnorm1(x)
        x = torch.relu(self.fc2(x))
        #x = self.drop(x)
        x = self.batchnorm2(x)
        x = torch.sigmoid(self.fc3(x))
        return x

In [50]:
input_size = x_train.shape[1]
model = BinaryClassifier(input_size)

In [51]:
criterion = nn.BCELoss()
optimizer = optim.AdamW(model.parameters(), lr=.0002)

In [52]:
def roc_score(y_true, y_pred):
    return roc_auc_score(y_true.detach().numpy(), y_pred.detach().numpy())

In [53]:
test_dataset = CustomDataset(x_test.values, y_test.values)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [54]:
EPOCHS = 100
device = 'cuda'
model.to(device)
for e in range(1, EPOCHS+1):
    epoch_loss = 0
    epoch_roc = 0
    model.train()
    for X_batch, y_batch in trainloader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()

        y_pred = model(X_batch)

        loss = criterion(y_pred, y_batch.unsqueeze(1))
        roc = roc_score(y_batch.cpu().unsqueeze(1), y_pred.cpu())

        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        epoch_roc += roc.item()

    if e % 5 == 0:
        test_roc = 0
        model.eval()
        for X_batch, y_batch in trainloader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            test_pred = model(X_batch)
            roc = roc_score(y_batch.cpu().unsqueeze(1), test_pred.cpu())
            test_roc += roc.item()
        print(f'Epoch {e+0:03}: | Loss: {epoch_loss/len(trainloader):.5f} | ROC: {epoch_roc/len(trainloader):.3f} | TEST ROC: {test_roc/len(trainloader):.3f}' )

Epoch 005: | Loss: 0.51377 | ROC: 0.815 | TEST ROC: 0.800
Epoch 010: | Loss: 0.45765 | ROC: 0.825 | TEST ROC: 0.831
Epoch 015: | Loss: 0.43893 | ROC: 0.831 | TEST ROC: 0.819
Epoch 020: | Loss: 0.42834 | ROC: 0.842 | TEST ROC: 0.844
Epoch 025: | Loss: 0.42347 | ROC: 0.843 | TEST ROC: 0.838
Epoch 030: | Loss: 0.42227 | ROC: 0.846 | TEST ROC: 0.819
Epoch 035: | Loss: 0.42135 | ROC: 0.846 | TEST ROC: 0.789
Epoch 040: | Loss: 0.41976 | ROC: 0.847 | TEST ROC: 0.846
Epoch 045: | Loss: 0.41842 | ROC: 0.850 | TEST ROC: 0.809
Epoch 050: | Loss: 0.41742 | ROC: 0.848 | TEST ROC: 0.826
Epoch 055: | Loss: 0.41805 | ROC: 0.847 | TEST ROC: 0.830
Epoch 060: | Loss: 0.41704 | ROC: 0.851 | TEST ROC: 0.851
Epoch 065: | Loss: 0.41568 | ROC: 0.849 | TEST ROC: 0.854
Epoch 070: | Loss: 0.41496 | ROC: 0.852 | TEST ROC: 0.831
Epoch 075: | Loss: 0.41504 | ROC: 0.853 | TEST ROC: 0.851
Epoch 080: | Loss: 0.41759 | ROC: 0.850 | TEST ROC: 0.825
Epoch 085: | Loss: 0.41496 | ROC: 0.852 | TEST ROC: 0.750
Epoch 090: | L

In [55]:
model.eval()
test_pred = model(torch.tensor(x_test.values, dtype=torch.float32).to(device))
test_pred = np.rint(test_pred.detach().to('cpu').numpy())

In [56]:
accuracy_score(y_test, test_pred)

0.6247334754797441

Точность получилась чуть меньше, чем у бустингов, хотя ROC_AUC повыше.

In [57]:
import shap

In [58]:
batch = next(iter(test_loader))
data, _ = batch

In [59]:
e = shap.DeepExplainer(model, data[:55].to(device))

In [60]:
shap_values = e.shap_values(data[55:])

Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.


In [61]:
shap.initjs()
shap.force_plot(e.expected_value, shap_values, x_test.columns)

In [62]:
shap.initjs()
shap.summary_plot(shap_values, data[55:], x_test.columns)

No data for colormapping provided via 'c'. Parameters 'vmin', 'vmax' will be ignored
Matplotlib is currently using module://matplotlib_inline.backend_inline, which is a non-GUI backend, so cannot show the figure.


Почему-то последний график не выводится, ещё попытался посмотреть *captum*, но не

In [98]:
style = pd.DataFrame(shap_values*10**10, columns=x_test.columns).round(2).style.background_gradient(cmap='Blues').format(precision=2)
style

,gender,SeniorCitizen,Partner,Dependents,PaperlessBilling,MonthlyCharges,TotalCharges,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,MultipleLines,Type_One year,Type_Two year,PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,InternetService_DSL,InternetService_Fiber optic
0,-0.00,-3.47,0.38,-5.28,0.00,-0.57,0.01,0.00,3.34,0.00,5.63,-8.87,0.00,-4.75,-0.00,0.00,0.00,-5.84,0.00,0.00,-0.00
1,-0.00,-3.47,0.38,0.00,-11.14,-2.15,-0.04,0.00,-0.00,9.32,5.63,-8.87,0.00,-4.75,-0.00,-0.81,-14.16,0.00,0.00,0.00,-0.00
2,-0.00,-3.47,0.38,-5.28,0.00,4.51,0.01,0.00,3.34,0.00,5.63,-0.00,0.00,0.00,-0.00,0.00,0.00,-5.84,0.00,-3.96,-4.52
3,-2.54,-3.47,-0.00,-5.28,0.00,5.98,0.01,0.00,3.34,0.00,5.63,-0.00,0.75,0.00,-0.00,0.00,0.00,0.00,0.00,-3.96,-4.52
4,-2.54,-3.47,0.38,0.00,-11.14,-3.54,-0.05,-5.72,-0.00,9.32,-0.00,-8.87,0.00,-4.75,6.66,0.00,-14.16,0.00,0.00,0.00,-0.00
5,-0.00,-3.47,0.38,-5.28,0.00,3.75,-0.02,0.00,-0.00,9.32,-0.00,-0.00,0.75,0.00,6.66,0.00,0.00,-5.84,0.00,-3.96,-4.52
6,-2.54,-3.47,0.38,-5.28,0.00,2.41,0.01,0.00,3.34,0.00,5.63,-0.00,0.75,0.00,-0.00,0.00,0.00,-5.84,0.00,0.00,-0.00
7,-2.54,-3.47,0.38,0.00,-11.14,9.24,0.00,0.00,3.34,0.00,5.63,-0.00,0.75,0.00,6.66,0.00,0.00,0.00,9.40,0.00,-4.52
8,-2.54,0.00,-0.00,0.00,0.00,1.93,0.01,0.00,-0.00,0.00,5.63,-0.00,0.75,0.00,-0.00,0.00,0.00,0.00,0.00,0.00,-0.00
